# Image Convolution Demo mit PyTorch

Dieses Notebook demonstriert verschiedene Convolution-Operationen mit PyTorch, einschließlich Bildladen, Definition von Filterkernen und Anwendung verschiedener Filter wie Binomial- und Sobel-Filter.

## 1. Import Required Libraries

Importiere PyTorch, torchvision, matplotlib, numpy und PIL für Bildverarbeitung und Visualisierung.

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import numpy as np
from PIL import Image, ImageOps

## 2. Load and Display Image

Lade ein Graustufenbild mit PIL und zeige es mit matplotlib an.

In [ ]:
# Bild einlesen
img = Image.open("SchwarzerKreis.png")
img = ImageOps.grayscale(img)
plt.figure(figsize=(8, 6))
plt.imshow(img, cmap='gray')
plt.title('Original Graustufenbild')
plt.axis('off')
plt.show()

## 3. Create Convolutional Layer

Erstelle einen PyTorch Conv2d Layer und untersuche sein State Dictionary.

In [ ]:
# Convolutional Layer anlegen
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, 
                   padding=1, dilation=1, bias=False, padding_mode='replicate')
print("Conv2D State Dictionary:")
print(conv2d.state_dict())

## 4. Define Filter Kernels

Definiere verschiedene Filterkerne einschließlich Gx, Gy (Sobel) und Binomial-Gewichte als numpy Arrays.

In [ ]:
# Filterkerne anlegen
GxWeights = np.array([[-0.25, 0, 0.25],
                     [-0.50, 0, 0.50],
                     [-0.25, 0, 0.25]], np.float32)

GyWeights = np.array([[-0.25, -0.50, -0.25],
                     [ 0,     0,    0],
                     [ 0.25,  0.50, 0.25]], np.float32)

BinomialWeights = np.array([[0.0625, 0.125, 0.0625],
                           [ 0.125, 0.250, 0.125],
                           [ 0.0625, 0.125, 0.0625]], np.float32)

print("Gx Weights (Sobel X):")
print(GxWeights)
print("\nGy Weights (Sobel Y):")
print(GyWeights)
print("\nBinomial Weights:")
print(BinomialWeights)

## 5. Apply Binomial Filter for Blurring

Erstelle eine Hilfsfunktion um Gewichte in den Convolutional Layer zu laden und wende einen Binomial-Filter für Bildunschärfe an.

In [ ]:
# Bildtransformation setzen
transform = transforms.ToTensor()

# Häufig verwendete Operationen zu einer Funktion formen
def WeightsToStateDict(weight, conv2d):
    weight = torch.from_numpy(weight)
    weight = torch.unsqueeze(torch.unsqueeze(weight, dim=0), dim=0)
    state_dict = conv2d.state_dict()
    state_dict['weight'] = weight
    conv2d.load_state_dict(state_dict)

# Binomialfilter setzen
weight = torch.unsqueeze(torch.unsqueeze(torch.from_numpy(BinomialWeights), dim=0), dim=0)
state_dict = conv2d.state_dict()
state_dict['weight'] = weight
conv2d.load_state_dict(state_dict)

# Binomial-Filter anwenden
imgBlured = conv2d(transform(img))

# Geblurrtes Bild anzeigen
result = imgBlured.permute(1, 2, 0)
plt.figure(figsize=(8, 6))
plt.imshow(result.detach().numpy(), cmap='gray')
plt.title('Geblurrtes Bild (Binomial Filter)')
plt.axis('off')
plt.show()

## 6. Apply Sobel Filters for Edge Detection

Konfiguriere einen Multi-Channel Convolutional Layer um sowohl Gx als auch Gy Sobel-Filter gleichzeitig für Kantenerkennung anzuwenden.

In [ ]:
# Kombiniere Gx und Gy Gewichte für Multi-Channel Convolution
weights = np.zeros((2, 3, 3))
weights[0, :, :] = GxWeights
weights[1, :, :] = GyWeights
print("Combined weights shape:", weights.shape)
print("Combined weights:")
print(weights)

# Erstelle Conv2d Layer mit 2 Output-Channels
conv2d = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, 
                   padding=1, dilation=1, bias=False, padding_mode='replicate')

# Lade die Sobel-Gewichte
state_dict = conv2d.state_dict()
state_dict['weight'] = torch.unsqueeze(torch.from_numpy(weights), dim=1)
conv2d.load_state_dict(state_dict)

# Wende Sobel-Filter auf das geblurrte Bild an
imgSobel = conv2d(imgBlured)
print("Sobel output shape:", imgSobel.shape)

## 7. Visualize Results

Zeige die gefilterten Bilder mit matplotlib an, um das Original, geblurrte und kantenerkannte Versionen zu vergleichen.

In [ ]:
# Visualisiere Gx und Gy Ergebnisse
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original Bild
axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('Original Bild')
axes[0, 0].axis('off')

# Geblurrtes Bild
axes[0, 1].imshow(result.detach().numpy(), cmap='gray')
axes[0, 1].set_title('Geblurrtes Bild (Binomial Filter)')
axes[0, 1].axis('off')

# Gx (horizontale Kanten)
gx_result = imgSobel[0, 0].detach().numpy()
axes[1, 0].imshow(gx_result, cmap='gray')
axes[1, 0].set_title('Gx - Horizontale Kanten (Sobel)')
axes[1, 0].axis('off')

# Gy (vertikale Kanten)
gy_result = imgSobel[0, 1].detach().numpy()
axes[1, 1].imshow(gy_result, cmap='gray')
axes[1, 1].set_title('Gy - Vertikale Kanten (Sobel)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# Berechne und zeige die Gradient-Magnitude
gradient_magnitude = np.sqrt(gx_result**2 + gy_result**2)
plt.figure(figsize=(8, 6))
plt.imshow(gradient_magnitude, cmap='gray')
plt.title('Gradient Magnitude (Kombinierte Kantenstärke)')
plt.axis('off')
plt.show()